In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from datetime import datetime
from sklearn.preprocessing import LabelEncoder



In [26]:
df_original = pd.read_csv("train.csv")

df = df_original.copy()
df.head(2)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999


In [15]:
df['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [16]:
null_columns = ["clean_title","accident","fuel_type"]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [18]:
df_original = pd.read_csv("train.csv")


In [19]:
# Handle clean_title Null values

df['clean_title'] = df['clean_title'].fillna('No')
df['clean_title'] = df['clean_title'].map({
    'Yes': 1,
    'No' : 0
})
df['clean_title'].unique()

array([1, 0])

In [20]:
# Handle accident Null values and mapping (0,1,-1)
df['accident'] = df['accident'].fillna('none reported')

df['accident'] = df['accident'].replace({
    'none reported': 0,
    'at least 1 accident or damage reported': 1,
})

df['accident'].unique()


array(['None reported', 'At least 1 accident or damage reported', 0],
      dtype=object)

In [21]:
# Handle fuel_type Null values and mapping

df['fuel_type'] = df['fuel_type'].replace(['–', 'Electric'], np.nan)
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])

# frequency encoding 



df['fuel_type'] = df['fuel_type'].map({
    'Gasoline' : 'Standard',
    'E85 Flex Fuel' : 'Standard',
    'Diesel': 'Premium',
    'Hybrid': 'Premium',
    'Plug-In Hybrid': 'Premium',
    'Electric' : 'Electric',
    'Unknown': 'Standard'
})


le = LabelEncoder()
df['fuel_type']  =le.fit_transform(df['fuel_type'])



df['fuel_type'].unique()

array([1, 0, 2])

In [22]:
# get the age of the car 

current_year = datetime.now().year
df['car_age'] = current_year - df['model_year']

df['car_age'].unique()

array([18, 23,  8,  4,  7,  9,  5, 10, 14, 12,  2,  6, 13, 11, 17, 16,  3,
       22, 20, 24, 19, 25, 15, 21, 28, 27, 26, 31, 32, 29, 30,  1, 51, 33])

In [27]:
df['engine'].unique()

array(['172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel',
       '252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel',
       '320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability', ...,
       '78.0HP 1.2L 3 Cylinder Engine Gasoline Fuel',
       '139.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas',
       '313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas'],
      shape=(1117,), dtype=object)

In [36]:
# Handle engine column (extract into 3 new columns)
#  HP, Capacity , cylinders

df['horse_power'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)

df['capacity'] = df['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)

df['cylinders'] = df['engine'].str.extract(r'(?:V)?(\d+)\s*(?:Cylinder|(?=\s|$))')


print (df['cylinders'].isnull().sum())

df['capacity'] = df['capacity'].fillna(df.groupby(['cylinders'])['capacity'].transform('median'))
df['capacity'] = df['capacity'].fillna(df['capacity'].median())


df['cylinders'] = df['cylinders'].astype(float)
df['cylinders'] = df['cylinders'].fillna(df['cylinders'].mode()[0])

nulls = df['horse_power'].isnull()
if nulls.sum() > 0:
    avg_hp_per_liter_cylinder = (df['horse_power'] / (df['capacity'] * df['cylinders'])).median()
    
    df.loc[nulls, 'horse_power'] = (
        df.loc[nulls, 'capacity'] * 
        df.loc[nulls, 'cylinders'] * 
        avg_hp_per_liter_cylinder
    )







6706


In [25]:
df['transmission'].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',
